In [1]:
# 1️⃣ 라이브러리 불러오기
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 2️⃣ 데이터셋 준비
BATCH_SIZE = 64

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 3️⃣ 학습/평가 함수 정의
def train(model, loader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            output = model(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(loader):.4f}')

def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            output = model(images)
            pred = output.argmax(dim=1)
            correct += (pred == labels).sum().item()
            total += labels.size(0)
    print(f'Accuracy: {correct/total:.4f}')

# 4️⃣ 원래 CNN 모델 정의 (MaxPool 포함)
class MNISTCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, 3, 1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.1)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, 1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.1)
        )
        self.fc = nn.Sequential(
            nn.Linear(64*5*5, 256),
            nn.ReLU(),
            nn.Linear(256, 10),
            nn.LogSoftmax(dim=1)
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

model = MNISTCNN().to(device)
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print("==== CNN Original Model Training ====")
train(model, train_loader, criterion, optimizer, epochs=5)
print("==== CNN Original Model Evaluation ====")
evaluate(model, test_loader)

# 5️⃣ MaxPool 제거 CNN 모델 정의
class MNISTCNNNoPool(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, 3, 1),
            nn.ReLU(),
            # nn.MaxPool2d(2, 2),  # 주석 처리
            nn.Dropout(0.1)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, 1),
            nn.ReLU(),
            # nn.MaxPool2d(2, 2),  # 주석 처리
            nn.Dropout(0.1)
        )
        # feature map 크기 계산 자동화
        dummy_input = torch.zeros(1, 1, 28, 28)
        dummy_output = self.layer2(self.layer1(dummy_input))
        n_features = dummy_output.numel()  # 64*24*24 처럼 자동 계산

        self.fc = nn.Sequential(
            nn.Linear(n_features, 256),
            nn.ReLU(),
            nn.Linear(256, 10),
            nn.LogSoftmax(dim=1)
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

model_no_pool = MNISTCNNNoPool().to(device)
optimizer_no_pool = optim.Adam(model_no_pool.parameters(), lr=0.001)

print("\n==== CNN No MaxPool Model Training ====")
train(model_no_pool, train_loader, criterion, optimizer_no_pool, epochs=5)
print("==== CNN No MaxPool Model Evaluation ====")
evaluate(model_no_pool, test_loader)


100%|██████████| 9.91M/9.91M [00:01<00:00, 6.08MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 160kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.51MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.81MB/s]


==== CNN Original Model Training ====
Epoch 1/5, Loss: 0.1498
Epoch 2/5, Loss: 0.0469
Epoch 3/5, Loss: 0.0339
Epoch 4/5, Loss: 0.0270
Epoch 5/5, Loss: 0.0204
==== CNN Original Model Evaluation ====
Accuracy: 0.9913

==== CNN No MaxPool Model Training ====
Epoch 1/5, Loss: 0.1335
Epoch 2/5, Loss: 0.0381
Epoch 3/5, Loss: 0.0229
Epoch 4/5, Loss: 0.0140
Epoch 5/5, Loss: 0.0095
==== CNN No MaxPool Model Evaluation ====
Accuracy: 0.9873
